# Summary

Generate adjancency matrices for the Protherm training set directly from PDBs.

---

# Imports

In [1]:
%run _imports.ipynb

Setting the PACKAGE_VERSION environment variable.
Setting the DOCS_SECRET_KEY environment variable.
Setting the PYTHON_VERSION environment variable.
Setting the SPARK_MASTER environment variable.
Setting the SPARK_ARGS environment variable.
Setting the DB_TYPE environment variable.
Setting the DB_PORT environment variable.


2018-01-17 16:44:59.923491


# Parameters

In [2]:
NOTEBOOK_NAME = 'protherm_dataset'
NOTEBOOK_PATH = Path(NOTEBOOK_NAME)

NOTEBOOK_PATH.mkdir(parents=True, exist_ok=True)

In [3]:
DATABIN_PATH = Path(os.environ['DATABIN_DIR'])

# Load data

In [4]:
ROSETTA_RESULTS = {}

with pd.HDFStore(DATABIN_PATH.joinpath('elapsam_feature_engineering/v0.1.0/rosetta.h5').as_posix(), 'r') as store:
    for key in store:
        ROSETTA_RESULTS[key.strip('/')] = store[key][:]

In [5]:
ROSETTA_RESULTS.keys()

dict_keys(['cartesian_ddg-beta_nov15_cart-1', 'cartesian_ddg-beta_nov16_cart-1', 'cartesian_ddg-score12_cart-1', 'cartesian_ddg-talaris2013_cart-1', 'cartesian_ddg-talaris2014_cart-1', 'ddg_monomer-soft_rep_design-1'])

In [6]:
ROSETTA_RESULTS['cartesian_ddg-talaris2014_cart-1'].head()

,filename-wt,pdb_chain,mutation,ddg,ddg_exp
Index,,,,,
0,/home/kimlab2/database_data/biological-data-warehouse/elapsam_feature_engineering/notebooks/elaspic_standalone/proth...,A,G44S,-0.633667,-0.53
1,/home/kimlab2/database_data/biological-data-warehouse/elapsam_feature_engineering/notebooks/elaspic_standalone/proth...,A,A120M,-0.188000,-0.20
2,/home/kimlab2/database_data/biological-data-warehouse/elapsam_feature_engineering/notebooks/elaspic_standalone/proth...,A,A116N,0.114000,0.17
3,/home/kimlab2/database_data/biological-data-warehouse/elapsam_feature_engineering/notebooks/elaspic_standalone/proth...,A,A122Q,0.508667,-0.24
4,/home/kimlab2/database_data/biological-data-warehouse/elapsam_feature_engineering/notebooks/elaspic_standalone/proth...,A,A123Q,-0.577667,-0.22


In [7]:
rosetta_results_df = None

for key, df in ROSETTA_RESULTS.items():
    df = df.rename(columns={'ddg': key})
    if rosetta_results_df is None:
        rosetta_results_df = df
    else:
        assert (rosetta_results_df['ddg_exp'].values == df['ddg_exp'].values).all()
        rosetta_results_df = rosetta_results_df.merge(
            df.drop('ddg_exp', axis=1), on=['filename-wt', 'pdb_chain', 'mutation'], how='outer')
    
display(rosetta_results_df.head())
print(rosetta_results_df.shape)

,filename-wt,pdb_chain,mutation,cartesian_ddg-beta_nov15_cart-1,ddg_exp,cartesian_ddg-beta_nov16_cart-1,cartesian_ddg-score12_cart-1,cartesian_ddg-talaris2013_cart-1,cartesian_ddg-talaris2014_cart-1,ddg_monomer-soft_rep_design-1
0,/home/kimlab2/database_data/biological-data-warehouse/elapsam_feature_engineering/notebooks/elaspic_standalone/proth...,A,G44S,-1.808667,-0.53,-0.701000,0.088000,-0.289667,-0.633667,-2.384
1,/home/kimlab2/database_data/biological-data-warehouse/elapsam_feature_engineering/notebooks/elaspic_standalone/proth...,A,A120M,2.617667,-0.20,0.354000,0.560000,-0.069000,-0.188000,2.472
2,/home/kimlab2/database_data/biological-data-warehouse/elapsam_feature_engineering/notebooks/elaspic_standalone/proth...,A,A116N,0.502667,0.17,-0.131333,0.419000,0.392000,0.114000,0.652
3,/home/kimlab2/database_data/biological-data-warehouse/elapsam_feature_engineering/notebooks/elaspic_standalone/proth...,A,A122Q,1.760333,-0.24,1.999000,3.205667,0.605667,0.508667,12.540
4,/home/kimlab2/database_data/biological-data-warehouse/elapsam_feature_engineering/notebooks/elaspic_standalone/proth...,A,A123Q,0.110333,-0.22,-1.153000,1.313000,-0.334333,-0.577667,-3.730


(3471, 10)


## Copy structures

In [8]:
STRUCTURE_PATH = NOTEBOOK_PATH.joinpath('structures')
STRUCTURE_PATH.mkdir(exist_ok=True)

In [9]:
# for file in STRUCTURE_PATH.glob('*'):
#     os.remove(file)

In [10]:
os.listdir(STRUCTURE_PATH)

[]

In [12]:
def get_local_filename(filename):
    return STRUCTURE_PATH.joinpath(op.basename(filename)).absolute().as_posix()

get_local_filename(rosetta_results_df['filename-wt'].iloc[0])

'/home/kimlab2/database_data/biological-data-warehouse/adjacency-net/notebooks/protherm_dataset/structures/pdb107l.entA_0--G44S-wt.pdb'

In [13]:
file_list = rosetta_results_df['filename-wt'].drop_duplicates().tolist()

for filename in tqdm.tqdm_notebook(file_list, total=len(file_list)):
    new_filename = STRUCTURE_PATH.joinpath(op.basename(filename))
    shutil.copy(filename, get_local_filename(filename))

rosetta_results_df['local-filename-wt'] = rosetta_results_df['filename-wt'].apply(get_local_filename)

HBox(children=(IntProgress(value=0, max=3471), HTML(value='')))

# Process data

## Extract adjacencies

In [28]:
from kmtools import structure_tools


def get_interaction_dataset(structure_file, R_CUTOFF):
    """Copied from "datapkg/pdb-analysis/notebooks/extract_pdb_interactions.ipynb"
    """
    try:
        structure = kmbio.PDB.load(structure_file, bioassembly_id=1, use_auth_id=False)
        bioassembly_id = 1
    except (kmbio.PDB.exceptions.BioassemblyError, KeyError) as e:
        logger.info("Could not construct bioassembly for file '%s'", structure_file)
        structure = kmbio.PDB.load(structure_file, bioassembly_id=0, use_auth_id=False)
        bioassembly_id = 0

    interactions = structure_tools.get_interactions(structure, r_cutoff=R_CUTOFF, interchain=True)
    interactions_core, interactions_interface = structure_tools.process_interactions(interactions)

    # Group interactions by chain / chan pair
    interactions_core_aggbychain = structure_tools.process_interactions_core(
        structure, interactions_core)
    interactions_interface_aggbychain = structure_tools.process_interactions_interface(
        structure, interactions_interface)

    # Drop duplicate rows?
    interactions_core, interactions_core_aggbychain = \
        structure_tools.drop_duplicates_core(interactions_core, interactions_core_aggbychain)
    interactions_interface, interactions_interface_aggbychain = \
        structure_tools.drop_duplicates_interface(interactions_interface,
                                                  interactions_interface_aggbychain)

    return (interactions_core, interactions_core_aggbychain, interactions_interface,
            interactions_interface_aggbychain, bioassembly_id)

In [59]:
kmbio.PDB.core.atom.logger.setLevel(logging.WARNING)

In [63]:
R_CUTOFF = 5

filenames = rosetta_results_df['local-filename-wt'].drop_duplicates().tolist()

adjacency_dfs = []
for filename in tqdm.tqdm_notebook(filenames, total=len(filenames)):
    df = get_interaction_dataset(filename, R_CUTOFF)[1]
    df['local-filename-wt'] = filename
    adjacency_dfs.append(df)

HBox(children=(IntProgress(value=0, max=3471), HTML(value='')))

/home/kimlab1/strokach/working/kmbio/kmbio/PDB/io/loaders.py:128: UserWarning: Not all arguments where used during the call to _get_parser! (kwargs = {'use_auth_id': False})
  kwargs))
Could not construct bioassembly for file '/home/kimlab2/database_data/biological-data-warehouse/adjacency-net/notebooks/protherm_dataset/structures/pdb107l.entA_0--G44S-wt.pdb'
Could not construct bioassembly for file '/home/kimlab2/database_data/biological-data-warehouse/adjacency-net/notebooks/protherm_dataset/structures/pdb160l.entA_0--A120M-wt.pdb'
Could not construct bioassembly for file '/home/kimlab2/database_data/biological-data-warehouse/adjacency-net/notebooks/protherm_dataset/structures/pdb161l.entA_0--A116N-wt.pdb'
Could not construct bioassembly for file '/home/kimlab2/database_data/biological-data-warehouse/adjacency-net/notebooks/protherm_dataset/structures/pdb162l.entA_0--A122Q-wt.pdb'
Could not construct bioassembly for file '/home/kimlab2/database_data/biological-data-warehouse/adjacenc

Could not construct bioassembly for file '/home/kimlab2/database_data/biological-data-warehouse/adjacency-net/notebooks/protherm_dataset/structures/pdb1a2p.entA_0--E71W-wt.pdb'
Exception in thread Thread-19:
Traceback (most recent call last):
  File "/home/kimlab1/strokach/anaconda3/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/home/kimlab1/strokach/anaconda3/lib/python3.6/site-packages/tqdm/_tqdm.py", line 144, in run
    for instance in self.tqdm_cls._instances:
  File "/home/kimlab1/strokach/anaconda3/lib/python3.6/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration

Could not construct bioassembly for file '/home/kimlab2/database_data/biological-data-warehouse/adjacency-net/notebooks/protherm_dataset/structures/pdb1a2p.entA_0--F5L-wt.pdb'


Could not construct bioassembly for file '/home/kimlab2/database_data/biological-data-warehouse/adjacency-net/notebooks/protherm_dataset/structures/pdb1a2p.entA_0--G32A-wt.pdb'
Could not construct bioassembly for file '/home/kimlab2/database_data/biological-data-warehouse/adjacency-net/notebooks/protherm_dataset/structures/pdb1a2p.entA_0--G32D-wt.pdb'
Could not construct bioassembly for file '/home/kimlab2/database_data/biological-data-warehouse/adjacency-net/notebooks/protherm_dataset/structures/pdb1a2p.entA_0--G32H-wt.pdb'
Could not construct bioassembly for file '/home/kimlab2/database_data/biological-data-warehouse/adjacency-net/notebooks/protherm_dataset/structures/pdb1a2p.entA_0--G32K-wt.pdb'
Could not construct bioassembly for file '/home/kimlab2/database_data/biological-data-warehouse/adjacency-net/notebooks/protherm_dataset/structures/pdb1a2p.entA_0--G32N-wt.pdb'
Could not construct bioassembly for file '/home/kimlab2/database_data/biological-data-warehouse/adjacency-net/noteb

Could not construct bioassembly for file '/home/kimlab2/database_data/biological-data-warehouse/adjacency-net/notebooks/protherm_dataset/structures/pdb1a2p.entA_0--K60R-wt.pdb'
Could not construct bioassembly for file '/home/kimlab2/database_data/biological-data-warehouse/adjacency-net/notebooks/protherm_dataset/structures/pdb1a2p.entA_0--K64A-wt.pdb'
Could not construct bioassembly for file '/home/kimlab2/database_data/biological-data-warehouse/adjacency-net/notebooks/protherm_dataset/structures/pdb1a2p.entA_0--L12A-wt.pdb'
Could not construct bioassembly for file '/home/kimlab2/database_data/biological-data-warehouse/adjacency-net/notebooks/protherm_dataset/structures/pdb1a2p.entA_0--L31Q-wt.pdb'
Could not construct bioassembly for file '/home/kimlab2/database_data/biological-data-warehouse/adjacency-net/notebooks/protherm_dataset/structures/pdb1a2p.entA_0--L87G-wt.pdb'
Could not construct bioassembly for file '/home/kimlab2/database_data/biological-data-warehouse/adjacency-net/noteb

...

Could not construct bioassembly for file '/home/kimlab2/database_data/biological-data-warehouse/adjacency-net/notebooks/protherm_dataset/structures/pdb2rn2.entA_0--R27A-wt.pdb'
Could not construct bioassembly for file '/home/kimlab2/database_data/biological-data-warehouse/adjacency-net/notebooks/protherm_dataset/structures/pdb2rn2.entA_0--R41C-wt.pdb'
Could not construct bioassembly for file '/home/kimlab2/database_data/biological-data-warehouse/adjacency-net/notebooks/protherm_dataset/structures/pdb2rn2.entA_0--S68A-wt.pdb'
Could not construct bioassembly for file '/home/kimlab2/database_data/biological-data-warehouse/adjacency-net/notebooks/protherm_dataset/structures/pdb2rn2.entA_0--S68G-wt.pdb'
Could not construct bioassembly for file '/home/kimlab2/database_data/biological-data-warehouse/adjacency-net/notebooks/protherm_dataset/structures/pdb2rn2.entA_0--S68L-wt.pdb'
Could not construct bioassembly for file '/home/kimlab2/database_data/biological-data-warehouse/adjacency-net/noteb

Could not construct bioassembly for file '/home/kimlab2/database_data/biological-data-warehouse/adjacency-net/notebooks/protherm_dataset/structures/pdb3chy.entA_0--A87G-wt.pdb'
Could not construct bioassembly for file '/home/kimlab2/database_data/biological-data-warehouse/adjacency-net/notebooks/protherm_dataset/structures/pdb3chy.entA_0--A89G-wt.pdb'
Could not construct bioassembly for file '/home/kimlab2/database_data/biological-data-warehouse/adjacency-net/notebooks/protherm_dataset/structures/pdb3chy.entA_0--A98G-wt.pdb'
Could not construct bioassembly for file '/home/kimlab2/database_data/biological-data-warehouse/adjacency-net/notebooks/protherm_dataset/structures/pdb3chy.entA_0--D11A-wt.pdb'
Could not construct bioassembly for file '/home/kimlab2/database_data/biological-data-warehouse/adjacency-net/notebooks/protherm_dataset/structures/pdb3chy.entA_0--D12A-wt.pdb'
Could not construct bioassembly for file '/home/kimlab2/database_data/biological-data-warehouse/adjacency-net/noteb

Could not construct bioassembly for file '/home/kimlab2/database_data/biological-data-warehouse/adjacency-net/notebooks/protherm_dataset/structures/pdb3pgk.entA_0--W308Y-wt.pdb'
Could not construct bioassembly for file '/home/kimlab2/database_data/biological-data-warehouse/adjacency-net/notebooks/protherm_dataset/structures/pdb3pgk.entA_0--W333F-wt.pdb'
Could not construct bioassembly for file '/home/kimlab2/database_data/biological-data-warehouse/adjacency-net/notebooks/protherm_dataset/structures/pdb3pgk.entA_0--W333Y-wt.pdb'
Could not construct bioassembly for file '/home/kimlab2/database_data/biological-data-warehouse/adjacency-net/notebooks/protherm_dataset/structures/pdb3ssi.entA_0--M98A-wt.pdb'
Could not construct bioassembly for file '/home/kimlab2/database_data/biological-data-warehouse/adjacency-net/notebooks/protherm_dataset/structures/pdb3ssi.entA_0--M98G-wt.pdb'
Could not construct bioassembly for file '/home/kimlab2/database_data/biological-data-warehouse/adjacency-net/no

Could not construct bioassembly for file '/home/kimlab2/database_data/biological-data-warehouse/adjacency-net/notebooks/protherm_dataset/structures/pdb4lyz.entA_0--D101S-wt.pdb'
Could not construct bioassembly for file '/home/kimlab2/database_data/biological-data-warehouse/adjacency-net/notebooks/protherm_dataset/structures/pdb4lyz.entA_0--E35A-wt.pdb'
Could not construct bioassembly for file '/home/kimlab2/database_data/biological-data-warehouse/adjacency-net/notebooks/protherm_dataset/structures/pdb4lyz.entA_0--E35Q-wt.pdb'
Could not construct bioassembly for file '/home/kimlab2/database_data/biological-data-warehouse/adjacency-net/notebooks/protherm_dataset/structures/pdb4lyz.entA_0--F34Y-wt.pdb'
Could not construct bioassembly for file '/home/kimlab2/database_data/biological-data-warehouse/adjacency-net/notebooks/protherm_dataset/structures/pdb4lyz.entA_0--F3Y-wt.pdb'
Could not construct bioassembly for file '/home/kimlab2/database_data/biological-data-warehouse/adjacency-net/noteb

Could not construct bioassembly for file '/home/kimlab2/database_data/biological-data-warehouse/adjacency-net/notebooks/protherm_dataset/structures/pdb4lyz.entA_0--W108Y-wt.pdb'
Could not construct bioassembly for file '/home/kimlab2/database_data/biological-data-warehouse/adjacency-net/notebooks/protherm_dataset/structures/pdb5azu.entA_0--A82G-wt.pdb'
Could not construct bioassembly for file '/home/kimlab2/database_data/biological-data-warehouse/adjacency-net/notebooks/protherm_dataset/structures/pdb5azu.entA_0--F110A-wt.pdb'
Could not construct bioassembly for file '/home/kimlab2/database_data/biological-data-warehouse/adjacency-net/notebooks/protherm_dataset/structures/pdb5azu.entA_0--F110S-wt.pdb'
Could not construct bioassembly for file '/home/kimlab2/database_data/biological-data-warehouse/adjacency-net/notebooks/protherm_dataset/structures/pdb5azu.entA_0--F97A-wt.pdb'
Could not construct bioassembly for file '/home/kimlab2/database_data/biological-data-warehouse/adjacency-net/no

In [66]:
adjacency_df = pd.concat(adjacency_dfs, axis=0, ignore_index=True)

In [71]:
adjacency_df['residue_idx_1_corrected'] = pd.Series(
    adjacency_df['residue_pair'].apply(lambda lst: [t[0] for t in lst]))
adjacency_df['residue_idx_2_corrected'] = pd.Series(
    adjacency_df['residue_pair'].apply(lambda lst: [t[1] for t in lst]))

In [72]:
adjacency_df = adjacency_df.drop('residue_pair', axis=1)

In [73]:
pq.write_table(
    pa.Table.from_pandas(adjacency_df, preserve_index=False),
    NOTEBOOK_PATH.joinpath('protherm_adjacencies.parquet').as_posix(),
    version='2.0', flavor='spark'
)

## Combine

In [76]:
rosetta_results_df.head(2)

,filename-wt,pdb_chain,mutation,cartesian_ddg-beta_nov15_cart-1,ddg_exp,cartesian_ddg-beta_nov16_cart-1,cartesian_ddg-score12_cart-1,cartesian_ddg-talaris2013_cart-1,cartesian_ddg-talaris2014_cart-1,ddg_monomer-soft_rep_design-1,local-filename-wt
0,/home/kimlab2/database_data/biological-data-warehouse/elapsam_feature_engineering/notebooks/elaspic_standalone/proth...,A,G44S,-1.808667,-0.53,-0.701,0.088,-0.289667,-0.633667,-2.384,/home/kimlab2/database_data/biological-data-warehouse/adjacency-net/notebooks/protherm_dataset/structures/pdb107l.en...
1,/home/kimlab2/database_data/biological-data-warehouse/elapsam_feature_engineering/notebooks/elaspic_standalone/proth...,A,A120M,2.617667,-0.20,0.354,0.560,-0.069000,-0.188000,2.472,/home/kimlab2/database_data/biological-data-warehouse/adjacency-net/notebooks/protherm_dataset/structures/pdb160l.en...


In [77]:
adjacency_df.head(2)

,structure_id,model_id,chain_id,protein_sequence,protein_sequence_hash,residue_sequence,residue_pair_hash,local-filename-wt,residue_idx_1_corrected,residue_idx_2_corrected
0,107l,0,A,MNIFEMLRIDEGLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAKGELDKAIGRNTNGVITKDEAEKLFNQDVDAAVRGILRNAKLKPVYDSLDAVRRAALINMVFQMGETGVAGFTN...,702F6C915C7242D7,"MET,ASN,ILE,PHE,GLU,MET,LEU,ARG,ILE,ASP,GLU,GLY,LEU,ARG,LEU,LYS,ILE,TYR,LYS,ASP,THR,GLU,GLY,TYR,TYR,THR,ILE,GLY,ILE,...",b0f2bc3f2dd1e9ac1537d7124c11285b,/home/kimlab2/database_data/biological-data-warehouse/adjacency-net/notebooks/protherm_dataset/structures/pdb107l.en...,"[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4...","[1, 2, 4, 5, 8, 157, 160, 0, 2, 3, 4, 5, 157, 0, 1, 3, 4, 5, 6, 7, 66, 70, 96, 99, 157, 1, 2, 4, 5, 6, 7, 66, 67, 70..."
1,160l,0,A,MNIFEMLRIDEGLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAKSELDKAIGRNTNGVITKDEAEKLFNQDVDAAVRGILRNAKLKPVYDSLDAVRRAALINMVFQMGETGVAGFTN...,942F635E748E4DBA,"MET,ASN,ILE,PHE,GLU,MET,LEU,ARG,ILE,ASP,GLU,GLY,LEU,ARG,LEU,LYS,ILE,TYR,LYS,ASP,THR,GLU,GLY,TYR,TYR,THR,ILE,GLY,ILE,...",1d757a99b0ce97c16ec50b2eb05298a1,/home/kimlab2/database_data/biological-data-warehouse/adjacency-net/notebooks/protherm_dataset/structures/pdb160l.en...,"[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4...","[1, 2, 4, 5, 8, 157, 160, 0, 2, 3, 4, 5, 6, 157, 0, 1, 3, 4, 5, 6, 7, 66, 70, 96, 99, 157, 1, 2, 4, 5, 6, 7, 8, 66, ..."


In [84]:
protherm_validaton_dataset = (
    rosetta_results_df
    .rename(columns={'pdb_chain': 'chain_id'})
    .merge(adjacency_df, how='left', on=['local-filename-wt', 'chain_id'])
)

In [85]:
protherm_validaton_dataset.head()

,filename-wt,chain_id,mutation,cartesian_ddg-beta_nov15_cart-1,ddg_exp,cartesian_ddg-beta_nov16_cart-1,cartesian_ddg-score12_cart-1,cartesian_ddg-talaris2013_cart-1,cartesian_ddg-talaris2014_cart-1,ddg_monomer-soft_rep_design-1,local-filename-wt,structure_id,model_id,protein_sequence,protein_sequence_hash,residue_sequence,residue_pair_hash,residue_idx_1_corrected,residue_idx_2_corrected
0,/home/kimlab2/database_data/biological-data-warehouse/elapsam_feature_engineering/notebooks/elaspic_standalone/proth...,A,G44S,-1.808667,-0.53,-0.701000,0.088000,-0.289667,-0.633667,-2.384,/home/kimlab2/database_data/biological-data-warehouse/adjacency-net/notebooks/protherm_dataset/structures/pdb107l.en...,107l,0,MNIFEMLRIDEGLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAKGELDKAIGRNTNGVITKDEAEKLFNQDVDAAVRGILRNAKLKPVYDSLDAVRRAALINMVFQMGETGVAGFTN...,702F6C915C7242D7,"MET,ASN,ILE,PHE,GLU,MET,LEU,ARG,ILE,ASP,GLU,GLY,LEU,ARG,LEU,LYS,ILE,TYR,LYS,ASP,THR,GLU,GLY,TYR,TYR,THR,ILE,GLY,ILE,...",b0f2bc3f2dd1e9ac1537d7124c11285b,"[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4...","[1, 2, 4, 5, 8, 157, 160, 0, 2, 3, 4, 5, 157, 0, 1, 3, 4, 5, 6, 7, 66, 70, 96, 99, 157, 1, 2, 4, 5, 6, 7, 66, 67, 70..."
1,/home/kimlab2/database_data/biological-data-warehouse/elapsam_feature_engineering/notebooks/elaspic_standalone/proth...,A,A120M,2.617667,-0.20,0.354000,0.560000,-0.069000,-0.188000,2.472,/home/kimlab2/database_data/biological-data-warehouse/adjacency-net/notebooks/protherm_dataset/structures/pdb160l.en...,160l,0,MNIFEMLRIDEGLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAKSELDKAIGRNTNGVITKDEAEKLFNQDVDAAVRGILRNAKLKPVYDSLDAVRRAALINMVFQMGETGVAGFTN...,942F635E748E4DBA,"MET,ASN,ILE,PHE,GLU,MET,LEU,ARG,ILE,ASP,GLU,GLY,LEU,ARG,LEU,LYS,ILE,TYR,LYS,ASP,THR,GLU,GLY,TYR,TYR,THR,ILE,GLY,ILE,...",1d757a99b0ce97c16ec50b2eb05298a1,"[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4...","[1, 2, 4, 5, 8, 157, 160, 0, 2, 3, 4, 5, 6, 157, 0, 1, 3, 4, 5, 6, 7, 66, 70, 96, 99, 157, 1, 2, 4, 5, 6, 7, 8, 66, ..."
2,/home/kimlab2/database_data/biological-data-warehouse/elapsam_feature_engineering/notebooks/elaspic_standalone/proth...,A,A116N,0.502667,0.17,-0.131333,0.419000,0.392000,0.114000,0.652,/home/kimlab2/database_data/biological-data-warehouse/adjacency-net/notebooks/protherm_dataset/structures/pdb161l.en...,161l,0,MNIFEMLRIDEGLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAKSELDKAIGRNTNGVITKDEAEKLFNQDVDAAVRGILRNAKLKPVYDSLDAVRRAALINMVFQMGETGVAGFTA...,0F49BC9148724186,"MET,ASN,ILE,PHE,GLU,MET,LEU,ARG,ILE,ASP,GLU,GLY,LEU,ARG,LEU,LYS,ILE,TYR,LYS,ASP,THR,GLU,GLY,TYR,TYR,THR,ILE,GLY,ILE,...",7e0057d9bfd050d637179e5f409cdb72,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4...","[1, 2, 4, 5, 8, 157, 160, 161, 0, 2, 3, 4, 5, 157, 0, 1, 3, 4, 5, 6, 7, 66, 70, 74, 96, 99, 157, 1, 2, 4, 5, 6, 7, 8..."
3,/home/kimlab2/database_data/biological-data-warehouse/elapsam_feature_engineering/notebooks/elaspic_standalone/proth...,A,A122Q,1.760333,-0.24,1.999000,3.205667,0.605667,0.508667,12.540,/home/kimlab2/database_data/biological-data-warehouse/adjacency-net/notebooks/protherm_dataset/structures/pdb162l.en...,162l,0,MNIFEMLRIDEGLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAKSELDKAIGRNTNGVITKDEAEKLFNQDVDAAVRGILRNAKLKPVYDSLDAVRRAALINMVFQMGETGVAGFTN...,813B3DC148724DBA,"MET,ASN,ILE,PHE,GLU,MET,LEU,ARG,ILE,ASP,GLU,GLY,LEU,ARG,LEU,LYS,ILE,TYR,LYS,ASP,THR,GLU,GLY,TYR,TYR,THR,ILE,GLY,ILE,...",06c025f9d9a2857eea757fd4b5c691fe,"[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4...","[1, 2, 4, 5, 8, 157, 160, 0, 2, 3, 4, 5, 6, 157, 0, 1, 3, 4, 5, 6, 7, 66, 70, 96, 99, 100, 157, 1, 2, 4, 5, 6, 7, 66..."
4,/home/kimlab2/database_data/biological-data-warehouse/elapsam_feature_engineering/notebooks/elaspic_standalone/proth...,A,A123Q,0.110333,-0.22,-1.153000,1.313000,-0.334333,-0.577667,-3.730,/home/kimlab

In [87]:
assert len(rosetta_results_df) == len(protherm_validaton_dataset)

In [88]:
assert not protherm_validaton_dataset.isnull().any().any()

In [89]:
assert not any(c.endswith('_1') for c in protherm_validaton_dataset.columns)

In [94]:
def mutation_matches_sequence(mutation, sequence):
    return sequence[int(mutation[1:-1]) - 1] == mutation[0]


protherm_validaton_dataset['mutation_matches_sequence'] = (
    protherm_validaton_dataset[['mutation', 'protein_sequence']].apply(
        lambda s: mutation_matches_sequence(s.mutation, s.protein_sequence),
        axis=1)
)

assert protherm_validaton_dataset['mutation_matches_sequence'].all()

## Write

In [99]:
columns = [
    'structure_id', 'model_id', 'chain_id', 'qseq', 'qseq_mutation', 'ddg_exp', 
    'residue_idx_1_corrected', 'residue_idx_2_corrected'
]

In [102]:
df = (
    protherm_validaton_dataset
    .rename(columns={'protein_sequence': 'qseq', 'mutation': 'qseq_mutation'})
    [columns]
)

pq.write_table(
    pa.Table.from_pandas(df, preserve_index=False),
    NOTEBOOK_PATH.joinpath('protherm_validaton_dataset.parquet').as_posix(),
    version='2.0', flavor='spark'
)

# Explore

In [109]:
Counter(protherm_validaton_dataset['mutation'].str[0])

Counter({'A': 281,
         'C': 43,
         'D': 199,
         'E': 231,
         'F': 129,
         'G': 161,
         'H': 101,
         'I': 307,
         'K': 233,
         'L': 233,
         'M': 71,
         'N': 115,
         'P': 98,
         'Q': 75,
         'R': 127,
         'S': 149,
         'T': 273,
         'V': 392,
         'W': 62,
         'Y': 191})

In [110]:
Counter(protherm_validaton_dataset['mutation'].str[-1])

Counter({'A': 1040,
         'C': 76,
         'D': 93,
         'E': 82,
         'F': 208,
         'G': 380,
         'H': 60,
         'I': 139,
         'K': 104,
         'L': 178,
         'M': 84,
         'N': 96,
         'P': 60,
         'Q': 118,
         'R': 67,
         'S': 165,
         'T': 131,
         'V': 278,
         'W': 45,
         'Y': 67})

In [108]:
for gp_index, gp in protherm_validaton_dataset.groupby(['structure_id', 'model_id', 'chain_id']):
    display(gp.head())
#     break

,filename-wt,chain_id,mutation,cartesian_ddg-beta_nov15_cart-1,ddg_exp,cartesian_ddg-beta_nov16_cart-1,cartesian_ddg-score12_cart-1,cartesian_ddg-talaris2013_cart-1,cartesian_ddg-talaris2014_cart-1,ddg_monomer-soft_rep_design-1,local-filename-wt,structure_id,model_id,protein_sequence,protein_sequence_hash,residue_sequence,residue_pair_hash,residue_idx_1_corrected,residue_idx_2_corrected,mutation_matches_sequence
0,/home/kimlab2/database_data/biological-data-warehouse/elapsam_feature_engineering/notebooks/elaspic_standalone/proth...,A,G44S,-1.808667,-0.53,-0.701,0.088,-0.289667,-0.633667,-2.384,/home/kimlab2/database_data/biological-data-warehouse/adjacency-net/notebooks/protherm_dataset/structures/pdb107l.en...,107l,0,MNIFEMLRIDEGLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAKGELDKAIGRNTNGVITKDEAEKLFNQDVDAAVRGILRNAKLKPVYDSLDAVRRAALINMVFQMGETGVAGFTN...,702F6C915C7242D7,"MET,ASN,ILE,PHE,GLU,MET,LEU,ARG,ILE,ASP,GLU,GLY,LEU,ARG,LEU,LYS,ILE,TYR,LYS,ASP,THR,GLU,GLY,TYR,TYR,THR,ILE,GLY,ILE,...",b0f2bc3f2dd1e9ac1537d7124c11285b,"[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4...","[1, 2, 4, 5, 8, 157, 160, 0, 2, 3, 4, 5, 157, 0, 1, 3, 4, 5, 6, 7, 66, 70, 96, 99, 157, 1, 2, 4, 5, 6, 7, 66, 67, 70...",True


,filename-wt,chain_id,mutation,cartesian_ddg-beta_nov15_cart-1,ddg_exp,cartesian_ddg-beta_nov16_cart-1,cartesian_ddg-score12_cart-1,cartesian_ddg-talaris2013_cart-1,cartesian_ddg-talaris2014_cart-1,ddg_monomer-soft_rep_design-1,local-filename-wt,structure_id,model_id,protein_sequence,protein_sequence_hash,residue_sequence,residue_pair_hash,residue_idx_1_corrected,residue_idx_2_corrected,mutation_matches_sequence
1,/home/kimlab2/database_data/biological-data-warehouse/elapsam_feature_engineering/notebooks/elaspic_standalone/proth...,A,A120M,2.617667,-0.2,0.354,0.56,-0.069,-0.188,2.472,/home/kimlab2/database_data/biological-data-warehouse/adjacency-net/notebooks/protherm_dataset/structures/pdb160l.en...,160l,0,MNIFEMLRIDEGLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAKSELDKAIGRNTNGVITKDEAEKLFNQDVDAAVRGILRNAKLKPVYDSLDAVRRAALINMVFQMGETGVAGFTN...,942F635E748E4DBA,"MET,ASN,ILE,PHE,GLU,MET,LEU,ARG,ILE,ASP,GLU,GLY,LEU,ARG,LEU,LYS,ILE,TYR,LYS,ASP,THR,GLU,GLY,TYR,TYR,THR,ILE,GLY,ILE,...",1d757a99b0ce97c16ec50b2eb05298a1,"[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4...","[1, 2, 4, 5, 8, 157, 160, 0, 2, 3, 4, 5, 6, 157, 0, 1, 3, 4, 5, 6, 7, 66, 70, 96, 99, 157, 1, 2, 4, 5, 6, 7, 8, 66, ...",True


,filename-wt,chain_id,mutation,cartesian_ddg-beta_nov15_cart-1,ddg_exp,cartesian_ddg-beta_nov16_cart-1,cartesian_ddg-score12_cart-1,cartesian_ddg-talaris2013_cart-1,cartesian_ddg-talaris2014_cart-1,ddg_monomer-soft_rep_design-1,local-filename-wt,structure_id,model_id,protein_sequence,protein_sequence_hash,residue_sequence,residue_pair_hash,residue_idx_1_corrected,residue_idx_2_corrected,mutation_matches_sequence
2,/home/kimlab2/database_data/biological-data-warehouse/elapsam_feature_engineering/notebooks/elaspic_standalone/proth...,A,A116N,0.502667,0.17,-0.131333,0.419,0.392,0.114,0.652,/home/kimlab2/database_data/biological-data-warehouse/adjacency-net/notebooks/protherm_dataset/structures/pdb161l.en...,161l,0,MNIFEMLRIDEGLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAKSELDKAIGRNTNGVITKDEAEKLFNQDVDAAVRGILRNAKLKPVYDSLDAVRRAALINMVFQMGETGVAGFTA...,0F49BC9148724186,"MET,ASN,ILE,PHE,GLU,MET,LEU,ARG,ILE,ASP,GLU,GLY,LEU,ARG,LEU,LYS,ILE,TYR,LYS,ASP,THR,GLU,GLY,TYR,TYR,THR,ILE,GLY,ILE,...",7e0057d9bfd050d637179e5f409cdb72,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4...","[1, 2, 4, 5, 8, 157, 160, 161, 0, 2, 3, 4, 5, 157, 0, 1, 3, 4, 5, 6, 7, 66, 70, 74, 96, 99, 157, 1, 2, 4, 5, 6, 7, 8...",True


,filename-wt,chain_id,mutation,cartesian_ddg-beta_nov15_cart-1,ddg_exp,cartesian_ddg-beta_nov16_cart-1,cartesian_ddg-score12_cart-1,cartesian_ddg-talaris2013_cart-1,cartesian_ddg-talaris2014_cart-1,ddg_monomer-soft_rep_design-1,local-filename-wt,structure_id,model_id,protein_sequence,protein_sequence_hash,residue_sequence,residue_pair_hash,residue_idx_1_corrected,residue_idx_2_corrected,mutation_matches_sequence
3,/home/kimlab2/database_data/biological-data-warehouse/elapsam_feature_engineering/notebooks/elaspic_standalone/proth...,A,A122Q,1.760333,-0.24,1.999,3.205667,0.605667,0.508667,12.54,/home/kimlab2/database_data/biological-data-warehouse/adjacency-net/notebooks/protherm_dataset/structures/pdb162l.en...,162l,0,MNIFEMLRIDEGLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAKSELDKAIGRNTNGVITKDEAEKLFNQDVDAAVRGILRNAKLKPVYDSLDAVRRAALINMVFQMGETGVAGFTN...,813B3DC148724DBA,"MET,ASN,ILE,PHE,GLU,MET,LEU,ARG,ILE,ASP,GLU,GLY,LEU,ARG,LEU,LYS,ILE,TYR,LYS,ASP,THR,GLU,GLY,TYR,TYR,THR,ILE,GLY,ILE,...",06c025f9d9a2857eea757fd4b5c691fe,"[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4...","[1, 2, 4, 5, 8, 157, 160, 0, 2, 3, 4, 5, 6, 157, 0, 1, 3, 4, 5, 6, 7, 66, 70, 96, 99, 100, 157, 1, 2, 4, 5, 6, 7, 66...",True


,filename-wt,chain_id,mutation,cartesian_ddg-beta_nov15_cart-1,ddg_exp,cartesian_ddg-beta_nov16_cart-1,cartesian_ddg-score12_cart-1,cartesian_ddg-talaris2013_cart-1,cartesian_ddg-talaris2014_cart-1,ddg_monomer-soft_rep_design-1,local-filename-wt,structure_id,model_id,protein_sequence,protein_sequence_hash,residue_sequence,residue_pair_hash,residue_idx_1_corrected,residue_idx_2_corrected,mutation_matches_sequence
4,/home/kimlab2/database_data/biological-data-warehouse/elapsam_feature_engineering/notebooks/elaspic_standalone/proth...,A,A123Q,0.110333,-0.22,-1.153,1.313,-0.334333,-0.577667,-3.73,/home/kimlab2/database_data/biological-data-warehouse/adjacency-net/notebooks/protherm_dataset/structures/pdb163l.en...,163l,0,MNIFEMLRIDEGLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAKSELDKAIGRNTNGVITKDEAEKLFNQDVDAAVRGILRNAKLKPVYDSLDAVRRAALINMVFQMGETGVAGFTN...,607E3C9148724DA0,"MET,ASN,ILE,PHE,GLU,MET,LEU,ARG,ILE,ASP,GLU,GLY,LEU,ARG,LEU,LYS,ILE,TYR,LYS,ASP,THR,GLU,GLY,TYR,TYR,THR,ILE,GLY,ILE,...",18067f5788126458390e65ea0c71ffcf,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4...","[1, 2, 4, 5, 8, 157, 160, 161, 0, 2, 3, 4, 5, 6, 157, 0, 1, 3, 4, 5, 6, 7, 66, 70, 96, 99, 157, 1, 2, 4, 5, 6, 7, 8,...",True


...

,filename-wt,chain_id,mutation,cartesian_ddg-beta_nov15_cart-1,ddg_exp,cartesian_ddg-beta_nov16_cart-1,cartesian_ddg-score12_cart-1,cartesian_ddg-talaris2013_cart-1,cartesian_ddg-talaris2014_cart-1,ddg_monomer-soft_rep_design-1,local-filename-wt,structure_id,model_id,protein_sequence,protein_sequence_hash,residue_sequence,residue_pair_hash,residue_idx_1_corrected,residue_idx_2_corrected,mutation_matches_sequence
3360,/home/kimlab2/database_data/biological-data-warehouse/elapsam_feature_engineering/notebooks/elaspic_standalone/proth...,A,E43Y,-0.526333,-0.604135,0.512667,-4.048333,0.526333,-0.017667,0.357,/home/kimlab2/database_data/biological-data-warehouse/adjacency-net/notebooks/protherm_dataset/structures/pdb451c.en...,451c,0,EDPEVLFKNKGCVACHAIDTKMVGPAYKDVAAKFAGQAGAEAELAQRIKNGSQGVWGPIPMPPNAVSDDEAQTLAKWVLSQK,D81A88C6D7923A68,"GLU,ASP,PRO,GLU,VAL,LEU,PHE,LYS,ASN,LYS,GLY,CYS,VAL,ALA,CYS,HIS,ALA,ILE,ASP,THR,LYS,MET,VAL,GLY,PRO,ALA,TYR,LYS,ASP,...",26d327e22fbe25562b70b9ca72422cee,"[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5...","[1, 2, 4, 5, 69, 72, 0, 2, 3, 4, 5, 6, 72, 0, 1, 3, 4, 5, 6, 72, 73, 76, 1, 2, 4, 5, 6, 7, 76, 0, 1, 2, 3, 5, 6, 7, ...",True
3361,/home/kimlab2/database_data/biological-data-warehouse/elapsam_feature_engineering/notebooks/elaspic_standalone/proth...,A,E70Q,4.099000,0.350000,5.167000,-3.356667,1.037333,0.889667,4.021,/home/kimlab2/database_data/biological-data-warehouse/adjacency-net/notebooks/protherm_dataset/structures/pdb451c.en...,451c,0,EDPEVLFKNKGCVACHAIDTKMVGPAYKDVAAKFAGQAGAEAELAQRIKNGSQGVWGPIPMPPNAVSDDEAQTLAKWVLSQK,D81A88C6D7923A68,"GLU,ASP,PRO,GLU,VAL,LEU,PHE,LYS,ASN,LYS,GLY,CYS,VAL,ALA,CYS,HIS,ALA,ILE,ASP,THR,LYS,MET,VAL,GLY,PRO,ALA,TYR,LYS,ASP,...",26d327e22fbe25562b70b9ca72422cee,"[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5...","[1, 2, 4, 5, 69, 72, 0, 2, 3, 4, 5, 6, 72, 0, 1, 3, 4, 5, 6, 72, 73, 76, 1, 2, 4, 5, 6, 7, 76, 0, 1, 2, 3, 5, 6, 7, ...",True
3362,/home/kimlab2/database_data/biological-data-warehouse/elapsam_feature_engineering/notebooks/elaspic_standalone/proth...,A,E70V,5.588000,1.150000,3.898000,-4.009000,1.530333,1.578667,6.218,/home/kimlab2/database_data/biological-data-warehouse/adjacency-net/notebooks/protherm_dataset/structures/pdb451c.en...,451c,0,EDPEVLFKNKGCVACHAIDTKMVGPAYKDVAAKFAGQAGAEAELAQRIKNGSQGVWGPIPMPPNAVSDDEAQTLAKWVLSQK,D81A88C6D7923A68,"GLU,ASP,PRO,GLU,VAL,LEU,PHE,LYS,ASN,LYS,GLY,CYS,VAL,ALA,CYS,HIS,ALA,ILE,ASP,THR,LYS,MET,VAL,GLY,PRO,ALA,TYR,LYS,ASP,...",26d327e22fbe25562b70b9ca72422cee,"[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5...","[1, 2, 4, 5, 69, 72, 0, 2, 3, 4, 5, 6, 72, 0, 1, 3, 4, 5, 6, 72, 73, 76, 1, 2, 4, 5, 6, 7, 76, 0, 1, 2, 3, 5, 6, 7, ...",True
3363,/home/kimlab2/database_data/biological-data-warehouse/elapsam_feature_engineering/notebooks/elaspic_standalone/proth...,A,F34Y,0.624667,-1.352127,0.737667,0.656000,0.381667,0.014667,8.713,/home/kimlab2/database_data/biological-data-warehouse/adjacency-net/notebooks/protherm_dataset/structures/pdb451c.en...,451c,0,EDPEVLFKNKGCVACHAIDTKMVGPAYKDVAAKFAGQAGAEAELAQRIKNGSQGVWGPIPMPPNAVSDDEAQTLAKWVLSQK,D81A88C6D7923A68,"GLU,ASP,PRO,GLU,VAL,LEU,PHE,LYS,ASN,LYS,GLY,CYS,VAL,ALA,CYS,HIS,ALA,ILE,ASP,THR,LYS,MET,VAL,GLY,PRO,ALA,TYR,LYS,ASP,...",26d327e22fbe25562b70b9ca72422cee,"[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5...","[1, 2, 4, 5, 69, 72, 0, 2, 3, 4, 5, 6, 72, 0, 1, 3, 4, 5, 6, 72, 73, 76, 1, 2, 4, 5, 6, 7, 76, 0, 1, 2, 3, 5, 6, 7, ...",True
3364,/home/kimlab2/database_data/biological-data-warehouse/elapsam_feature_engineering/notebooks/elaspic_standalone/proth...,A,F7A,4.271000,-1.100000,6.159000,1.803000,4.644667,6.149000,1.410,/home/kimlab2/database_data/biological-data-warehouse/adjacency-net/notebooks/protherm_datase

,filename-wt,chain_id,mutation,cartesian_ddg-beta_nov15_cart-1,ddg_exp,cartesian_ddg-beta_nov16_cart-1,cartesian_ddg-score12_cart-1,cartesian_ddg-talaris2013_cart-1,cartesian_ddg-talaris2014_cart-1,ddg_monomer-soft_rep_design-1,local-filename-wt,structure_id,model_id,protein_sequence,protein_sequence_hash,residue_sequence,residue_pair_hash,residue_idx_1_corrected,residue_idx_2_corrected,mutation_matches_sequence
3370,/home/kimlab2/database_data/biological-data-warehouse/elapsam_feature_engineering/notebooks/elaspic_standalone/proth...,A,S93C,-2.073667,-1.35,-1.960000,-1.820333,-2.285667,1.041667,-0.303,/home/kimlab2/database_data/biological-data-warehouse/adjacency-net/notebooks/protherm_dataset/structures/pdb4blm.en...,4blm,0,DDFAKLEEQFDAKLGIFALDTGTNRTVAYRPDERFAFASTIKALTVGVLLQQKSIEDLNQRITYTRDDLVNYNPITEKHVDTGMTLKELADASLRYSDNAAQNLILKQIGGPESLK...,FDA4770D4C6D50F6,"ASP,ASP,PHE,ALA,LYS,LEU,GLU,GLU,GLN,PHE,ASP,ALA,LYS,LEU,GLY,ILE,PHE,ALA,LEU,ASP,THR,GLY,THR,ASN,ARG,THR,VAL,ALA,TYR,...",be9a117bf0f46f10a12584afbe1934ac,"[0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4...","[1, 2, 3, 4, 5, 0, 2, 3, 4, 5, 249, 0, 1, 3, 4, 5, 6, 13, 15, 26, 28, 246, 249, 250, 0, 1, 2, 4, 5, 6, 7, 28, 0, 1, ...",True
3371,/home/kimlab2/database_data/biological-data-warehouse/elapsam_feature_engineering/notebooks/elaspic_standalone/proth...,A,S230C,2.255000,1.20,0.905333,0.520333,1.497333,2.606000,1.411,/home/kimlab2/database_data/biological-data-warehouse/adjacency-net/notebooks/protherm_dataset/structures/pdb4blm.en...,4blm,0,DDFAKLEEQFDAKLGIFALDTGTNRTVAYRPDERFAFASTIKALTVGVLLQQKSIEDLNQRITYTRDDLVNYNPITEKHVDTGMTLKELADASLRYSDNAAQNLILKQIGGPESLK...,FDA4770D4C6D50F6,"ASP,ASP,PHE,ALA,LYS,LEU,GLU,GLU,GLN,PHE,ASP,ALA,LYS,LEU,GLY,ILE,PHE,ALA,LEU,ASP,THR,GLY,THR,ASN,ARG,THR,VAL,ALA,TYR,...",be9a117bf0f46f10a12584afbe1934ac,"[0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4...","[1, 2, 3, 4, 5, 0, 2, 3, 4, 5, 249, 0, 1, 3, 4, 5, 6, 13, 15, 26, 28, 246, 249, 250, 0, 1, 2, 4, 5, 6, 7, 28, 0, 1, ...",True


,filename-wt,chain_id,mutation,cartesian_ddg-beta_nov15_cart-1,ddg_exp,cartesian_ddg-beta_nov16_cart-1,cartesian_ddg-score12_cart-1,cartesian_ddg-talaris2013_cart-1,cartesian_ddg-talaris2014_cart-1,ddg_monomer-soft_rep_design-1,local-filename-wt,structure_id,model_id,protein_sequence,protein_sequence_hash,residue_sequence,residue_pair_hash,residue_idx_1_corrected,residue_idx_2_corrected,mutation_matches_sequence
3372,/home/kimlab2/database_data/biological-data-warehouse/elapsam_feature_engineering/notebooks/elaspic_standalone/proth...,A,A31I,3.378333,-1.400000,-1.952333,3.567000,1.138667,1.659000,77.614,/home/kimlab2/database_data/biological-data-warehouse/adjacency-net/notebooks/protherm_dataset/structures/pdb4lyz.en...,4lyz,0,KVFGRCELAAAMKRHGLDNYRGYSLGNWVCAAKFESNFNTQATNRNTDGSTDYGILQINSRWWCNDGRTPGSRNLCNIPCSALLSSDITASVNCAKKIVSDGNGMNAWVAWRNRCK...,601CC8BCCA275EBD,"LYS,VAL,PHE,GLY,ARG,CYS,GLU,LEU,ALA,ALA,ALA,MET,LYS,ARG,HIS,GLY,LEU,ASP,ASN,TYR,ARG,GLY,TYR,SER,LEU,GLY,ASN,TRP,VAL,...",2b707a725eace99296b28c93929e1bad,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3...","[1, 2, 6, 37, 38, 39, 40, 85, 0, 2, 3, 6, 36, 37, 38, 39, 40, 0, 1, 3, 4, 6, 7, 10, 36, 37, 38, 39, 54, 85, 87, 1, 2...",True
3373,/home/kimlab2/database_data/biological-data-warehouse/elapsam_feature_engineering/notebooks/elaspic_standalone/proth...,A,A31L,-4.131667,-1.800000,-9.636000,-0.250333,-3.334667,-3.867333,5.634,/home/kimlab2/database_data/biological-data-warehouse/adjacency-net/notebooks/protherm_dataset/structures/pdb4lyz.en...,4lyz,0,KVFGRCELAAAMKRHGLDNYRGYSLGNWVCAAKFESNFNTQATNRNTDGSTDYGILQINSRWWCNDGRTPGSRNLCNIPCSALLSSDITASVNCAKKIVSDGNGMNAWVAWRNRCK...,601CC8BCCA275EBD,"LYS,VAL,PHE,GLY,ARG,CYS,GLU,LEU,ALA,ALA,ALA,MET,LYS,ARG,HIS,GLY,LEU,ASP,ASN,TYR,ARG,GLY,TYR,SER,LEU,GLY,ASN,TRP,VAL,...",2b707a725eace99296b28c93929e1bad,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3...","[1, 2, 6, 37, 38, 39, 40, 85, 0, 2, 3, 6, 36, 37, 38, 39, 40, 0, 1, 3, 4, 6, 7, 10, 36, 37, 38, 39, 54, 85, 87, 1, 2...",True
3374,/home/kimlab2/database_data/biological-data-warehouse/elapsam_feature_engineering/notebooks/elaspic_standalone/proth...,A,A31V,0.058667,-1.200000,-3.802667,5.352000,1.933667,2.297667,77.916,/home/kimlab2/database_data/biological-data-warehouse/adjacency-net/notebooks/protherm_dataset/structures/pdb4lyz.en...,4lyz,0,KVFGRCELAAAMKRHGLDNYRGYSLGNWVCAAKFESNFNTQATNRNTDGSTDYGILQINSRWWCNDGRTPGSRNLCNIPCSALLSSDITASVNCAKKIVSDGNGMNAWVAWRNRCK...,601CC8BCCA275EBD,"LYS,VAL,PHE,GLY,ARG,CYS,GLU,LEU,ALA,ALA,ALA,MET,LYS,ARG,HIS,GLY,LEU,ASP,ASN,TYR,ARG,GLY,TYR,SER,LEU,GLY,ASN,TRP,VAL,...",2b707a725eace99296b28c93929e1bad,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3...","[1, 2, 6, 37, 38, 39, 40, 85, 0, 2, 3, 6, 36, 37, 38, 39, 40, 0, 1, 3, 4, 6, 7, 10, 36, 37, 38, 39, 54, 85, 87, 1, 2...",True
3375,/home/kimlab2/database_data/biological-data-warehouse/elapsam_feature_engineering/notebooks/elaspic_standalone/proth...,A,C94A,-4.130333,4.780076,4.620000,-2.473667,-0.001667,-0.002000,0.000,/home/kimlab2/database_data/biological-data-warehouse/adjacency-net/notebooks/protherm_dataset/structures/pdb4lyz.en...,4lyz,0,KVFGRCELAAAMKRHGLDNYRGYSLGNWVCAAKFESNFNTQATNRNTDGSTDYGILQINSRWWCNDGRTPGSRNLCNIPCSALLSSDITASVNCAKKIVSDGNGMNAWVAWRNRCK...,601CC8BCCA275EBD,"LYS,VAL,PHE,GLY,ARG,CYS,GLU,LEU,ALA,ALA,ALA,MET,LYS,ARG,HIS,GLY,LEU,ASP,ASN,TYR,ARG,GLY,TYR,SER,LEU,GLY,ASN,TRP,VAL,...",794dd75fbbf6197eb6f168d18ac6fb61,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3...","[1, 2, 6, 37, 38, 39, 40, 85, 0, 2, 3, 6, 36, 37, 38, 39, 40, 0, 1, 3, 4, 6, 7, 10, 36, 37, 38, 39, 54, 85, 87, 1, 2...",True
3376,/home/kimlab2/database_data/biological-data-warehouse/elapsam_feature_engineering/notebooks/elaspic_standalone/proth...,A,D

,filename-wt,chain_id,mutation,cartesian_ddg-beta_nov15_cart-1,ddg_exp,cartesian_ddg-beta_nov16_cart-1,cartesian_ddg-score12_cart-1,cartesian_ddg-talaris2013_cart-1,cartesian_ddg-talaris2014_cart-1,ddg_monomer-soft_rep_design-1,local-filename-wt,structure_id,model_id,protein_sequence,protein_sequence_hash,residue_sequence,residue_pair_hash,residue_idx_1_corrected,residue_idx_2_corrected,mutation_matches_sequence
3433,/home/kimlab2/database_data/biological-data-warehouse/elapsam_feature_engineering/notebooks/elaspic_standalone/proth...,A,A82G,7.705000,2.031529,7.335667,5.390000,3.161000,3.887667,6.580,/home/kimlab2/database_data/biological-data-warehouse/adjacency-net/notebooks/protherm_dataset/structures/pdb5azu.en...,5azu,0,AECSVDIQGNDQMQFNTNAITVDKSCKQFTVNLSHPGNLPKNVMGHNWVLSTAADMQGVVTDGMASGLDKDYLKPDDSRVIAHTKLIGSGEKDSVTFDVSKLKEGEQYMFFCTFPG...,B28E0136E0A74E0E,"ALA,GLU,CYS,SER,VAL,ASP,ILE,GLN,GLY,ASN,ASP,GLN,MET,GLN,PHE,ASN,THR,ASN,ALA,ILE,THR,VAL,ASP,LYS,SER,CYS,LYS,GLN,PHE,...",dfb429ef83fdbd6b3f367534fa8b62de,"[0, 0, 0, 0, 0, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5...","[1, 2, 3, 4, 5, 0, 2, 3, 0, 1, 3, 4, 25, 27, 28, 29, 0, 1, 2, 4, 5, 28, 29, 0, 2, 3, 5, 6, 19, 28, 29, 30, 31, 0, 3,...",True
3434,/home/kimlab2/database_data/biological-data-warehouse/elapsam_feature_engineering/notebooks/elaspic_standalone/proth...,A,F110A,11.444000,5.019080,14.483333,6.515000,6.820333,8.721333,12.071,/home/kimlab2/database_data/biological-data-warehouse/adjacency-net/notebooks/protherm_dataset/structures/pdb5azu.en...,5azu,0,AECSVDIQGNDQMQFNTNAITVDKSCKQFTVNLSHPGNLPKNVMGHNWVLSTAADMQGVVTDGMASGLDKDYLKPDDSRVIAHTKLIGSGEKDSVTFDVSKLKEGEQYMFFCTFPG...,B28E0136E0A74E0E,"ALA,GLU,CYS,SER,VAL,ASP,ILE,GLN,GLY,ASN,ASP,GLN,MET,GLN,PHE,ASN,THR,ASN,ALA,ILE,THR,VAL,ASP,LYS,SER,CYS,LYS,GLN,PHE,...",dfb429ef83fdbd6b3f367534fa8b62de,"[0, 0, 0, 0, 0, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5...","[1, 2, 3, 4, 5, 0, 2, 3, 0, 1, 3, 4, 25, 27, 28, 29, 0, 1, 2, 4, 5, 28, 29, 0, 2, 3, 5, 6, 19, 28, 29, 30, 31, 0, 3,...",True
3435,/home/kimlab2/database_data/biological-data-warehouse/elapsam_feature_engineering/notebooks/elaspic_standalone/proth...,A,F110S,13.253667,4.900000,17.827333,8.707000,8.091000,10.497667,-44.264,/home/kimlab2/database_data/biological-data-warehouse/adjacency-net/notebooks/protherm_dataset/structures/pdb5azu.en...,5azu,0,AECSVDIQGNDQMQFNTNAITVDKSCKQFTVNLSHPGNLPKNVMGHNWVLSTAADMQGVVTDGMASGLDKDYLKPDDSRVIAHTKLIGSGEKDSVTFDVSKLKEGEQYMFFCTFPG...,B28E0136E0A74E0E,"ALA,GLU,CYS,SER,VAL,ASP,ILE,GLN,GLY,ASN,ASP,GLN,MET,GLN,PHE,ASN,THR,ASN,ALA,ILE,THR,VAL,ASP,LYS,SER,CYS,LYS,GLN,PHE,...",dfb429ef83fdbd6b3f367534fa8b62de,"[0, 0, 0, 0, 0, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5...","[1, 2, 3, 4, 5, 0, 2, 3, 0, 1, 3, 4, 25, 27, 28, 29, 0, 1, 2, 4, 5, 28, 29, 0, 2, 3, 5, 6, 19, 28, 29, 30, 31, 0, 3,...",True
3436,/home/kimlab2/database_data/biological-data-warehouse/elapsam_feature_engineering/notebooks/elaspic_standalone/proth...,A,F97A,11.988000,2.629042,14.797333,9.324000,7.051000,8.977000,13.261,/home/kimlab2/database_data/biological-data-warehouse/adjacency-net/notebooks/protherm_dataset/structures/pdb5azu.en...,5azu,0,AECSVDIQGNDQMQFNTNAITVDKSCKQFTVNLSHPGNLPKNVMGHNWVLSTAADMQGVVTDGMASGLDKDYLKPDDSRVIAHTKLIGSGEKDSVTFDVSKLKEGEQYMFFCTFPG...,B28E0136E0A74E0E,"ALA,GLU,CYS,SER,VAL,ASP,ILE,GLN,GLY,ASN,ASP,GLN,MET,GLN,PHE,ASN,THR,ASN,ALA,ILE,THR,VAL,ASP,LYS,SER,CYS,LYS,GLN,PHE,...",dfb429ef83fdbd6b3f367534fa8b62de,"[0, 0, 0, 0, 0, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5...","[1, 2, 3, 4, 5, 0, 2, 3, 0, 1, 3, 4, 25, 27, 28, 29, 0, 1, 2, 4, 5, 28, 29, 0, 2, 3, 5, 6, 19, 28, 29, 30, 31, 0, 3,...",True
3437,/home/kimlab2/database_data/biological-data-warehouse/elapsam_feature_engineering/notebooks/elaspic_standalone/proth...,A,H117

,filename-wt,chain_id,mutation,cartesian_ddg-beta_nov15_cart-1,ddg_exp,cartesian_ddg-beta_nov16_cart-1,cartesian_ddg-score12_cart-1,cartesian_ddg-talaris2013_cart-1,cartesian_ddg-talaris2014_cart-1,ddg_monomer-soft_rep_design-1,local-filename-wt,structure_id,model_id,protein_sequence,protein_sequence_hash,residue_sequence,residue_pair_hash,residue_idx_1_corrected,residue_idx_2_corrected,mutation_matches_sequence
3461,/home/kimlab2/database_data/biological-data-warehouse/elapsam_feature_engineering/notebooks/elaspic_standalone/proth...,A,A36S,0.780000,-0.4,1.745333,2.232333,1.075333,0.987667,-3.668,/home/kimlab2/database_data/biological-data-warehouse/adjacency-net/notebooks/protherm_dataset/structures/pdb5cro.en...,5cro,0,MEQRITLKDYAMRFGQTKTAKDLGVYQSAINKAIHAGRKIFLTINADGSVYAEEVKPFPSN,23E4D47BD7D97E10,"MET,GLU,GLN,ARG,ILE,THR,LEU,LYS,ASP,TYR,ALA,MET,ARG,PHE,GLY,GLN,THR,LYS,THR,ALA,LYS,ASP,LEU,GLY,VAL,TYR,GLN,SER,ALA,...",738a7800a3fd0b1522fda24923fc209c,"[0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5...","[1, 2, 43, 45, 0, 2, 3, 42, 43, 44, 0, 1, 3, 4, 41, 42, 43, 1, 2, 4, 5, 40, 41, 42, 43, 54, 2, 3, 5, 6, 8, 9, 12, 39...",True
3462,/home/kimlab2/database_data/biological-data-warehouse/elapsam_feature_engineering/notebooks/elaspic_standalone/proth...,A,Q16L,-1.025000,-2.8,-2.591333,-1.441667,1.351000,0.582333,15.510,/home/kimlab2/database_data/biological-data-warehouse/adjacency-net/notebooks/protherm_dataset/structures/pdb5cro.en...,5cro,0,MEQRITLKDYAMRFGQTKTAKDLGVYQSAINKAIHAGRKIFLTINADGSVYAEEVKPFPSN,23E4D47BD7D97E10,"MET,GLU,GLN,ARG,ILE,THR,LEU,LYS,ASP,TYR,ALA,MET,ARG,PHE,GLY,GLN,THR,LYS,THR,ALA,LYS,ASP,LEU,GLY,VAL,TYR,GLN,SER,ALA,...",738a7800a3fd0b1522fda24923fc209c,"[0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5...","[1, 2, 43, 45, 0, 2, 3, 42, 43, 44, 0, 1, 3, 4, 41, 42, 43, 1, 2, 4, 5, 40, 41, 42, 43, 54, 2, 3, 5, 6, 8, 9, 12, 39...",True
3463,/home/kimlab2/database_data/biological-data-warehouse/elapsam_feature_engineering/notebooks/elaspic_standalone/proth...,A,Y26C,0.795667,-2.2,0.764667,-0.062000,-0.252000,0.226667,3.023,/home/kimlab2/database_data/biological-data-warehouse/adjacency-net/notebooks/protherm_dataset/structures/pdb5cro.en...,5cro,0,MEQRITLKDYAMRFGQTKTAKDLGVYQSAINKAIHAGRKIFLTINADGSVYAEEVKPFPSN,23E4D47BD7D97E10,"MET,GLU,GLN,ARG,ILE,THR,LEU,LYS,ASP,TYR,ALA,MET,ARG,PHE,GLY,GLN,THR,LYS,THR,ALA,LYS,ASP,LEU,GLY,VAL,TYR,GLN,SER,ALA,...",738a7800a3fd0b1522fda24923fc209c,"[0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5...","[1, 2, 43, 45, 0, 2, 3, 42, 43, 44, 0, 1, 3, 4, 41, 42, 43, 1, 2, 4, 5, 40, 41, 42, 43, 54, 2, 3, 5, 6, 8, 9, 12, 39...",True
3464,/home/kimlab2/database_data/biological-data-warehouse/elapsam_feature_engineering/notebooks/elaspic_standalone/proth...,A,Y26D,-4.740667,-2.7,-2.633000,0.144333,-0.705333,-0.433333,0.803,/home/kimlab2/database_data/biological-data-warehouse/adjacency-net/notebooks/protherm_dataset/structures/pdb5cro.en...,5cro,0,MEQRITLKDYAMRFGQTKTAKDLGVYQSAINKAIHAGRKIFLTINADGSVYAEEVKPFPSN,23E4D47BD7D97E10,"MET,GLU,GLN,ARG,ILE,THR,LEU,LYS,ASP,TYR,ALA,MET,ARG,PHE,GLY,GLN,THR,LYS,THR,ALA,LYS,ASP,LEU,GLY,VAL,TYR,GLN,SER,ALA,...",738a7800a3fd0b1522fda24923fc209c,"[0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5...","[1, 2, 43, 45, 0, 2, 3, 42, 43, 44, 0, 1, 3, 4, 41, 42, 43, 1, 2, 4, 5, 40, 41, 42, 43, 54, 2, 3, 5, 6, 8, 9, 12, 39...",True
3465,/home/kimlab2/database_data/biological-data-warehouse/elapsam_feature_engineering/notebooks/elaspic_standalone/proth...,A,Y26F,0.771667,-0.4,0.376000,0.027333,0.141000,0.384333,3.005,/home/kimlab2/database_data/biological-data-warehouse/adjacency-net/notebooks/protherm_dataset/structures/pdb5cro.en...,5cro,0,MEQRITLKDYAMRFGQTKTAKDLGVYQSAINKAIHAGRKIFLTINADGSVYAEEVKPFPSN,23E4

In [105]:
gp[0]

('107l', 0)